In [48]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from functools import reduce
from math import ceil, exp
from tqdm.auto import tqdm
from tabulate import tabulate



In [49]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return np.array(sets.toarray())

In [50]:
num_points = [100, 1_000, 5_000]
num_sets = num_points[0]
num_points = num_points[0]
density = [.3,.7]
density = density[0]

set_threashold = ceil(num_points/2)  #number of iteration 



In [51]:
x = make_set_covering_problem(num_points = num_points,num_sets= num_sets, density=density)

def check_problem_solvable(sets : sparse.lil_array, num_points):
    """Returns True if the problem is solvable, False otherwise"""  
    #sets = sets.toarray()  
    current = reduce(np.logical_or, [sets[i] for i in range(sets.shape[0]) ], np.array([False for _ in range(num_points)]) )   
    return np.all(current)
        
assert check_problem_solvable(x,num_points) == True , "The problem is not solvable"

# Hill Climbing

In [52]:
def tweak_hill_climbing(current_state : np.array,sets : np.array) -> np.array:  #this approach favor the exploitation 
    state = current_state.copy()
    min_convered = (np.sum([sets[i] for i in state])).argmin()
    adding = randint(1,3)
    for i in range(state.shape[0]):  #to avoid duplicates in the state
        index = randint(0, sets.shape[0]-1)
        if index in state:
            continue
        else : 
            adding -= 1
            if adding == 0:
                state[min_convered] = index
            else :
                state = np.append(state,index)
                break     
    return state

def covering(state : np.array
             ,sets : np.array
             ) : 
    """Returns the number of points covered by the given state"""
    current = reduce(np.logical_or, [sets[i] for i in state ], np.array([False for _ in range(sets.shape[0])]) )
    return np.sum(current) , 0

def covering_and_num_sets(state : np.array
             ,sets : np.array
             ): 
    current = reduce(np.logical_or, [sets[i] for i in state], np.array([False for _ in range(sets.shape[0])]) )
    return np.sum(current), len(state)

fitness = covering_and_num_sets

In [53]:
def tweak_hill_climbing_exploration(current_state : np.array,sets : np.array) -> np.array:  #this approach favor the exploration 
    state :np.array = current_state.copy()
    min_convered = (np.sum([sets[i] for i in state])).argmin()
    state = np.delete(state, min_convered)
    adding = randint(1, 5)
    added = 0

    for _ in range(state.shape[0]):  
        index = randint(0, sets.shape[0]-1)
        if index in state:
            continue
        else : 
            state = np.append(state, index)
            added += 1
        if added == adding:
                break        
    return state

In [54]:
inferior_limit = ceil(num_sets/500)
superior_limit = ceil(num_sets /100)
number_of_sets = randint(inferior_limit,superior_limit)
changes = 0
#crete number_of_sets indexes to take from the sets, withput duplicates
original_state =np.array(list(set([randint(0, num_sets) for _ in range(number_of_sets)])))
state = original_state.copy()
current_ev = None
with tqdm(total=None) as pbar:
    for i in range(set_threashold): 
        if current_ev and current_ev == num_points:
            break
        current_ev , nc_sets = fitness(state,x)
        next_state = tweak_hill_climbing(state,x)
        next_ev , nn_sets = fitness(next_state,x)
        if next_ev > current_ev or (next_ev == current_ev and nn_sets < nc_sets):
            state = next_state
            changes +=1
        pbar.update(1)
print("Sub optimal solution ", len(state) , "sets with ", fitness(state,x), "/", num_points ," points covered doing ", "fitness called", i-1, "times")
#print(reduce(np.logical_or, [x[i] for i in state ], np.array([False for _ in range(num_points)])))

9it [00:00, 9048.12it/s]

Sub optimal solution  9 sets with  (100, 9) / 100  points covered doing  fitness called 8 times


In [55]:
changes = 0
#crete number_of_sets indexes to take from the sets, withput duplicates
state = original_state.copy()
current_ev = None

with tqdm(total=None) as pbar:
    for i in range(set_threashold): 
        if current_ev and current_ev == num_points:
            break
        current_ev, nc_sets = fitness(state,x)
        next_state = tweak_hill_climbing_exploration(state,x)
        next_ev, nn_sets = fitness(next_state,x)
        if next_ev > current_ev or (next_ev == current_ev and nn_sets < nc_sets):
            state = next_state
            changes +=1
        pbar.update(1)
print("Sub optimal solution ", len(state) , "sets with ", fitness(state,x), "/", num_points ," points covered doing ", i, " iteration")
#print(reduce(np.logical_or, [x[i] for i in state ], np.array([False for _ in range(num_points)])))

50it [00:00, 15003.23it/s]

Sub optimal solution  1 sets with  (32, 1) / 100  points covered doing  49  iteration


# Simulated Annealing
accept also worse solution trying to go to a global optimum

In [56]:

def exponential_cooling(initial_temperature, cooling_rate, iteration):
    """
    Exponential cooling schedule for Simulated Annealing.
    
    :param initial_temperature: Initial temperature.
    :param cooling_rate: Rate at which temperature decreases (between 0 and 1).
    :param iteration: Current iteration.
    :return: Current temperature.
    """
    return initial_temperature * (cooling_rate ** iteration)
    #return initial_temperature * cooling_rate

def simulated_annealing(current_cost : int, new_cost : int , temp : float) -> bool: 
    try :
        value = exp(-(current_cost-new_cost)/temp) if temp != 0 else exp(-(current_cost-new_cost)/ float('inf'))
        rand = random()
        #print("value", value, "rand", rand)
        if rand < value : 
            return True
        return False
    except OverflowError:
        return False


def tweak_simulated_annealing(current_state : np.array, sets : np.array) -> np.array: 
    """Returns a neighbor of the given state"""
    state = current_state.copy()
    #add a random number of sets 
    #remove a random number of sets
    to_add = randint(1, 5)
    to_remove = randint(1, 5) 
    #print("to_add",to_add,"to_remove", to_remove)     
    added = 0
    removed = 0
    for _ in range(state.shape[0]):
        set_ = randint(0, sets.shape[0]-1)
        if set_ in state and removed < to_remove:
            removed+=1
            state = np.delete(state, np.where(state == set_))
        elif set_ not in state and added < to_add :
            state = np.append(state, set_)
            added += 1
        if added == to_add and removed == to_remove:
            break
    #print("previous :", len(current_state), "current :", len(state))
    return state
            

temperature = 10 #higher value -> exploration #lowe value -> exploitation 
cooling_rate = 0.95 
state = original_state.copy()
changes_up = np.zeros(set_threashold, dtype=int)
changes_down = 0
print(len(state))
current_ev = None
with tqdm(total=None) as pbar:
    for iteration in range(set_threashold): 
        if current_ev and current_ev == num_points:
            break
        current_ev , nc_sets = fitness(state,x)
        next_state = tweak_simulated_annealing(state,x)
        next_ev , nn_sets = fitness(next_state,x)
        temperature=exponential_cooling(temperature,cooling_rate,iteration)
        if next_ev> current_ev or (next_ev == current_ev and nn_sets < nc_sets): 
            state=next_state 
            changes_up[changes_down] +=1
        elif next_ev< current_ev  and simulated_annealing(current_ev,next_ev, temperature):
            #print("annealing")
            state = next_state
            changes_down +=1
        pbar.update(1)
print("Sub optimal solution ", len(state) , "sets with ", fitness(state,x), "/", num_points ," points covered doing ", changes_up[0:changes_down+1], " changes_up and " , iteration , "iteration")

1


7it [00:00, 4831.35it/s]

Sub optimal solution  14 sets with  (100, 14) / 100  points covered doing  [3 2]  changes_up and  7 iteration


# Tabu search

In [57]:
def tweak_tabu_search(state : np.array, sets: np.array, tabu_list : list): 
    """Returns a neighbor of the given state"""
    current_state = state.copy()
    #add a one random set
    min_convered = (np.sum([sets[i] for i in state])).argmin() 
    adding = randint(1,2)
    current_state = np.delete(current_state, min_convered)
    for i in range(sets.shape[0]):  #to avoid duplicates in the state
        set_ = randint(0, sets.shape[0]-1)
        if set_ in state:
            continue
        else : 
            tmp = np.append(state,set_)        

            if vector_in_list(tmp, tabu_list):
                #print("already visited")
                continue
            else :
                adding -= 1
                state = np.append(state,set_)
                if adding == 0:
                    break     
    return state
        
def vector_in_list(vector : np.array, list_ : list) -> bool:
    for v in list_:
        if np.array_equal(v, vector):
            print("vector")
            return True
    return False   
    

state = original_state.copy()
tabu_list = [state]
tabu_list_limit = 150 
current_ev = None
with tqdm(total=None) as pbar:
    for iteration in range(set_threashold): 
        if current_ev and current_ev == num_points:
            break
        current_ev, nc_sets = fitness(state,x)
        next_state = tweak_tabu_search(state,x,tabu_list)
        next_ev, nn_sets = fitness(next_state,x)
        if next_ev> current_ev or (next_ev == current_ev and nn_sets < nc_sets): 
            state=next_state 
            tabu_list.append(next_state)
        if len(tabu_list) > tabu_list_limit : 
            tabu_list.pop(0)
        pbar.update(1)
print("Sub optimal solution ", len(state) , "sets with ", fitness(state,x), "/", num_points ," points covered doing ", iteration, " iteration")


10it [00:00, 10015.05it/s]

Sub optimal solution  13 sets with  (100, 13) / 100  points covered doing  10  iteration


# Iterated Local Search

In [58]:
def new_starting_position(global_ : np.array, last_ : np.array):
    number_of_sets = randint(ceil(len(last_)/2),ceil(len(last_)))
    new_state = []
    for i in range(number_of_sets):
        new_state.append(randint(0, global_.shape[0]-1))
    return np.array(list(set(new_state)))
        

            

def tweak_iterated_local(current_state : np.array, sets: np.array):
    state = current_state.copy()
    min_convered = (np.sum([sets[i] for i in state])).argmin()
    adding = randint(1,2)
    for i in range(state.shape[0]):  #to avoid duplicates in the state
        index = randint(0, sets.shape[0]-1)
        if index in state:
            continue
        else : 
            adding -= 1
            if adding == 0:
                state[min_convered] = index
            else :
                state = np.append(state,index)
                break     
    return state


state = original_state.copy()
best_ev, nb_sets = fitness(state,x)
best_state = state.copy()
not_better = 0
threashold_not_better = ceil(set_threashold/10)
current_ev = 0
with tqdm(total=None) as pbar:
    for iteration in range(set_threashold): 
        if current_ev and current_ev == num_points:
            break
        if not_better >= threashold_not_better:
            #print("new starting position aftern not_better iterations" , not_better)
            #print("best_ev", best_ev, "nb_sets", nb_sets)
            state = new_starting_position(x ,state )
            not_better = 0
        
        current_ev, nc_sets = fitness(state,x)
        next_state = tweak_iterated_local(state,x)
        next_ev, nn_sets = fitness(next_state,x)
        if next_ev> current_ev or (next_ev == current_ev and nn_sets < nc_sets): 
            state=next_state 
            tabu_list.append(next_state)
            not_better = 0
        if best_ev < next_ev or (next_ev == best_ev and nn_sets < nb_sets):
            best_state = next_state.copy()
            best_ev = next_ev
            nb_sets = nn_sets
            not_better = 0

        if next_ev <= current_ev:
            not_better +=1

        
        pbar.update(1)
print("Sub optimal solution ", len(best_state) , "sets with ", fitness(best_state,x), "/", num_points ," points covered doing ", iteration, " iteration")


14it [00:00, 14041.19it/s]

Sub optimal solution  13 sets with  (100, 13) / 100  points covered doing  14  iteration


In [59]:
num_points = [100, 1_000, 5_000]
density = [.3,.7]

  #number of iteration 

record = []
for size in num_points:
    for dens in density : 
        inferior_limit = ceil(1)
        superior_limit = ceil(5)
        number_of_sets = randint(inferior_limit,superior_limit)
        set_threashold = ceil(size/2)
        problem = make_set_covering_problem(num_points = size,num_sets= size, density=dens)   
        original_state =np.array(list(set([randint(0, number_of_sets) for _ in range(size)])))

        state = original_state.copy()
        current_ev = None
        for i in range(set_threashold): 
            if current_ev and current_ev == size:
                break
            current_ev , nc_sets = fitness(state,problem)
            next_state = tweak_hill_climbing(state,problem)
            next_ev , nn_sets = fitness(next_state,problem)
            if next_ev > current_ev or (next_ev == current_ev and nn_sets < nc_sets):
                state = next_state
        record.append([size , dens, len(state), i])

headers = ["Parameter", "Density", "Solution Size", "Fitness Calls"]
print(tabulate(record, headers, tablefmt="pretty"))


record = []
for size in num_points:
    for dens in density : 
        inferior_limit = ceil(1)
        superior_limit = ceil(5)
        number_of_sets = randint(inferior_limit,superior_limit)
        set_threashold = ceil(size/2)
        problem = make_set_covering_problem(num_points = size,num_sets= size, density=dens)   
        original_state =np.array(list(set([randint(0, number_of_sets) for _ in range(size)])))

        state = original_state.copy()
        current_ev = None
        for i in range(set_threashold): 
            if current_ev and current_ev == size:
                break
            current_ev , nc_sets = fitness(state,problem)
            next_state = tweak_hill_climbing_exploration(state,problem)
            next_ev , nn_sets = fitness(next_state,problem)
            if next_ev > current_ev or (next_ev == current_ev and nn_sets < nc_sets):
                state = next_state
        record.append([size , dens, len(state), i])

headers = ["Parameter", "Density", "Solution Size", "Fitness Calls"]
print(tabulate(record, headers, tablefmt="pretty"))

        

+-----------+---------+---------------+---------------+
| Parameter | Density | Solution Size | Fitness Calls |
+-----------+---------+---------------+---------------+
|    100    |   0.3   |      11       |       9       |
|    100    |   0.7   |       6       |       2       |
|   1000    |   0.3   |      17       |      26       |
|   1000    |   0.7   |       7       |       5       |
|   5000    |   0.3   |      20       |      17       |
|   5000    |   0.7   |       8       |       4       |
+-----------+---------+---------------+---------------+
